# Exploring IGoR's Inferred Models

With IGoR is possible to infer a V(D)J recombination model given sequences data.
This models can be defined by the user or use the default models within IGoR to evaluate a model.
IGoR's models are specified in 2 files "model_parms.txt" and "model_marginals.txt"

* model_parms.txt: Contains parameters used in the model and the structure of the bayesian network.
* model_marginals.txt: Contains the NOT the probability marginals, it contains the conditional probabilities factorized in the bayesian network.

In this tutorial we gonna start with the IGoR's default models and use pygor3 package to make visualizations of these inferred models.

If you have IGoR installed in your computer (IGoR's version >= 1.4.1) pygor3 can detect automatically your default models directory.
Now, lets try to load a model for human species and tcr_beta chain

In [ ]:
import pygor3 as p3

In [ ]:
species="human"
chain="tcr_beta"
mdl = p3.IgorModel.load_default(species, chain)

In [ ]:
mdl.parms['d_gene']

In [ ]:
mdl.marginals['d_gene']

In [ ]:
mdl.xdata['d_gene']

Now, we load the information of the model in the mdl variable. 
Notice that once you load a model pygor3 outputs the path of the files used, and all information in both files is encapsulated in the variable "mdl"

In particular all the information of conditional probabilities are in mdl.xdata. mdl.xdata is a dictionary with xarray structures.

In [ ]:
mdl.xdata.keys()

## Structure of the Model

We can visualize the structure of the IGoR bayesian model by using plot_Bayes_network()

In [ ]:
mdl.plot_Bayes_network()

With this we can see the nickname of the events and explore the contained information.


To get the conditional probabilities of an specific event we could use mdl.xdata["event_nickname"] or simply
mdl["event_nickname"]

In [ ]:
mdl['j_choice']

The last steps returns a xarray structure with the information about the conditional probability P(j_choice|v_choice). From this you can get the seqs and labels of the gene events

In [ ]:
mdl['j_choice'].dims

In [ ]:
mdl['j_choice']['j_choice']

In [ ]:
mdl['j_choice']['lbl__j_choice']

In [ ]:
mdl['j_choice']['seq__j_choice']

To get specific probability values a python dictionary is used to query a specific probability.
Example: to get the J gene with id 0 given that V with id 2 has choosen.

In [ ]:
mdl['j_choice'][{'j_choice':0, 'v_choice':2}]

As you can see from the Bayesian network plot, j_choice has parents and childs, and this information is stored as an attribute in the xarray structure among other informations.

In [ ]:
mdl['j_choice'].attrs

Notice that this operation preserve the labels and sequence and id of the events.
From this point you can explore model as you which. For instance you can calculate the marginal probability of 'j_choice'.

As can be seen 'j_choice' parent is only 'v_choice' and because 'v_choice' has no parents then the marginal for this event should be

In [ ]:
Pmarginal_j_choice = mdl['j_choice'].dot(mdl['v_choice'])
Pmarginal_j_choice

In [ ]:
Pmarginal_j_choice.plot()

However, pygor3 calculates the marginals when a new model is load and is stored in the dictionary parameter Pmarginal and a way to plot it using the method plot_Event_Marginal

In [ ]:
mdl.Pmarginal['j_choice']

In [ ]:
mdl.plot_Event_Marginal('j_choice')

### Export models

Models can be exported to pandas dataframes in case you need it and then exporte it to a file manually or you can use the export-model pygor3 script

In [ ]:
df_ev = mdl.Pmarginal['v_choice'].to_dataframe(name='P')
df_ev.head()

## Plotting models

The plot_Event_Marginal can be personalized if an matplotlib axes object is pass in the paramter ax.

### Plotting GeneChoice marginals

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(18, 20))
grid = plt.GridSpec(2, 3, hspace=0.2, wspace=0.2)
grid
ax_v = fig.add_subplot(grid[0, :])
ax_j = fig.add_subplot(grid[1, 0:2])
ax_d = fig.add_subplot(grid[1, 2])

mdl.plot_Event_Marginal('v_choice', ax=ax_v)
mdl.plot_Event_Marginal('j_choice', ax=ax_j)
mdl.plot_Event_Marginal('d_gene', ax=ax_d)

### Plotting Insertions and deletions marginals

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 6))
mdl.plot_Event_Marginal('v_3_del', ax=ax[0], marker='o', label='V3')
mdl.plot_Event_Marginal('j_5_del', ax=ax[0], marker='x', label='J5')
mdl.plot_Event_Marginal('d_3_del', ax=ax[0], marker='s', label='D3')
mdl.plot_Event_Marginal('d_5_del', ax=ax[0], marker='s', label='D5')
ax[0].set_xlabel("Deletions")
ax[0].legend()

mdl.plot_Event_Marginal('vd_ins', ax=ax[1], marker='o', label='VD')
mdl.plot_Event_Marginal('dj_ins', ax=ax[1], marker='s', label='DJ')
ax[1].set_xlabel("Insertions")
ax[1].legend()

### Plotting DinucMarkov marginals

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(18,8))
mdl.plot_Event_Marginal('vd_dinucl', ax=ax[0])
mdl.plot_Event_Marginal('dj_dinucl', ax=ax[1])

### Conditional Probabilities

In [ ]:
event_nickname='d_3_del'
mdl[event_nickname]

In [ ]:
fig, ax = plt.subplots(figsize=(18,15))
event_nickname='j_choice'
mdl[event_nickname].plot(ax=ax, cmap='gnuplot2_r')
titulo = "$P($"+event_nickname+"$|$"+",".join(mdl[event_nickname].attrs['parents'])+"$)$"
ax.set_title(titulo)
ax.set_aspect('equal')

In [ ]:
event_nickname
# mdl[event_nickname].plot()

In [ ]:
import matplotlib.pyplot as plt
# mdl[event_nickname].dims
event_nickname='d_gene'
da = mdl[event_nickname]
fig, ax = plt.subplots(*da[event_nickname].shape, figsize=(10,20))

for ii, ev_realiz in enumerate(da[event_nickname]):
    #print(ev_realiz.values)
    da[{event_nickname: ev_realiz.values}].plot(ax=ax[ii], cmap='gnuplot2_r')
    titulo = "$P($" + event_nickname + "$ = $ " +ev_realiz["lbl__"+event_nickname].values + " $|$" + ",".join(da.attrs['parents']) + "$)$"
    ax[ii].set_title(titulo)
    

In [ ]:
mdl['v_3_del']

In [ ]:
df_entropy_decomposition = mdl.get_df_entropy_decomposition()
df_entropy_decomposition

In [ ]:
mdl.plot_recombination_entropy()

In [ ]:
len(mdl.parms['v_choice'])
import numpy as np
10**( 5.3*np.log10(2) )

In [ ]:
mdl.export_csv('tmp')
# libreoffice tmp_

In [ ]:
mdl.export_plot_Pconditionals('tmp_CP')

In [ ]:
mdl.export_plot_Pmarginals('tmp_MP')

In [ ]:
P_VD = mdl_hb.get_P_joint(['v_choice', 'd_gene'])
P_VD

In [ ]:
P_VD.plot()

In [ ]:
da_mi = mdl.get_mutual_information

In [ ]:
mdl.plot_mutual_information(da_mi)